In [ ]:
import requests
from pathlib import Path
from zipfile import ZipFile
import geopandas as gpd
import numpy as np
import plotly.graph_objects as go
import json

# get the shape file...
url = "https://avoidatastr.blob.core.windows.net/avoindata/AvoinData/9_Kartat/PKS%20postinumeroalueet/Shp/PKS_postinumeroalueet_2021_shp.zip"

fn = Path.cwd().joinpath(url.split("/")[-1])
if not fn.exists():
    r = requests.get(url, stream=True)
    with open(fn, "wb") as f:
        for chunk in r.raw.stream(1024, decode_content=False):
            if chunk:
                f.write(chunk)

zfile = ZipFile(fn)
zfile.extractall()

# open it...
geodf = gpd.read_file(list(Path.cwd().glob("PKS*.shp"))[0])
geodf["rand"] = np.random.randint(1, 100, len(geodf))

# shape file is a different CRS,  change to lon/lat GPS co-ordinates
geodf = geodf.to_crs("WGS84").set_index("Posno")


fig = go.Figure(go.Choroplethmapbox(geojson=json.loads(geodf.to_json()), 
                                    locations=geodf.index, z=geodf['rand'],
                                    colorscale="Viridis", marker_line_width=.5))

fig.update_layout(mapbox_style="open-street-map",
                        height = 1000,
                        autosize=True,
                        margin={"r":0,"t":0,"l":0,"b":0},
                        paper_bgcolor='#303030',
                        plot_bgcolor='#303030',
                        mapbox=dict(center=dict(lat=60.1699, lon=24.9384),zoom=9),
                        )